#Theano

Theano is a python library and an optimizing compiler, which allows to define, optimizw and compute mathematical expressions effectively using multidimensional arrays.

Features:

* integration with NumPy
* effective computation of gradient (can automatically build expressions to compute gradient)
* fast and stable optimization (can recognize numerical inaccurate expressions and compute them using more stable algorithms)
* clear using GPU
* dynamic C++ code generation

In [ ]:
%pylab inline

## Import 
* theano
* theano.tensor

In [ ]:
import numpy
import theano.tensor as T
import theano

## Symbolic expessions for tensors
Theano’s strength is in expressing symbolic calculations involving tensors. There are many types of symbolic expressions for tensors:

* scalar
* vector
* matrix
* tensor
* ...

In [ ]:
# create two vectors and one scaler
x = T.vector() 
y = T.vector()
alpha = T.scalar()

#### compute 
$x + \alpha y + (\sum x_i, ... ,\sum x_i)^T$

In [ ]:
# define mathematical expression (you can use any function, which you use for NumPy arrays)
z = x + alpha * y + T.sum(x)

## Compile defined expression
`theano.function`

Returns a callable object that will calculate outputs from inputs

In [ ]:
# input variables, output expessions
compiled_expr = theano.function([x, y, alpha], z)

## Compute compiled expression

In [ ]:
x_val = numpy.arange(10)
y_val = numpy.arange(10)
alpha_val = 0.5

In [ ]:
compiled_expr(x_val, y_val, alpha_val)

## Exercise

* compute $(<x, \alpha y> + <\beta x, y>)^2$

In [ ]:
# create two vectors and one scaler


## Names for expressions

In [ ]:
# define 
x = T.vector(name='x') 
y = T.vector(name='y')
alpha = T.scalar(name='a')

In [ ]:
z = alpha * x * T.log(y) 

### `theano.printing.pprint()`
Theano provides the functions `theano.printing.pprint()` and `theano.printing.debugprint()` to print a graph to the terminal before or after compilation. `pprint()` is more compact and math-like, `debugprint()` is more verbose. Theano also provides `pydotprint()` that creates an image of the function. 

In [ ]:
theano.pprint(x)

In [ ]:
theano.pprint(z)

In [ ]:
compiled_expr = theano.function(inputs=[x, y, alpha], outputs=[z], name='function')

In [ ]:
theano.printing.debugprint(compiled_expr)

In [ ]:
theano.printing.pydotprint(compiled_expr, outfile="graph.png", var_with_name_simple=True)

## Gradient

`grad` returns symbolic gradients for one or more variables with respect to some cost.

### 1D 

In [ ]:
x = T.scalar(name='x') 
func = T.log(x) * T.sinh(x)
func_prime = T.grad(func, x)

In [ ]:
theano.pprint(func_prime)

#### Check yourself that this expession indeed gradient!!!

In [ ]:
func_prime_function = theano.function([x], func_prime)

In [ ]:
x = numpy.linspace(0.1, 2, 100)
plot([func_prime_function(point) for point in x])

### ND

In [ ]:
x = T.vector(name='x') 
func = T.sum(x * x)
func_prime = T.grad(func, x)

In [ ]:
theano.pprint(func_prime)

In [ ]:
func_prime_function = theano.function([x], func_prime)

In [ ]:
x1 = numpy.linspace(0.1, 2, 10)
x2 = numpy.linspace(0.1, 2, 10)
[func_prime_function(numpy.array([point1, point2])) for point1, point2 in zip(x1, x2)]

#### Check yourself that this result is correct!!!

## Shared variables
`theano.shared` returns a SharedVariable variable, initialized with a copy or reference of `value`.

Variable with Storage that is shared between functions that it appears in. These variables are meant to be created by registered shared constructors (see `shared_constructor()`).
The user-friendly constructor is `shared()`

In [ ]:
w = theano.shared(numpy.arange(10, dtype=float), name='weight')
x = T.vector('x')
func = theano.function([x], T.sum(x * w))
func_grad = theano.function([x], T.grad(T.sum(x * w), x))

In [ ]:
func_grad(numpy.arange(10) * 0.)

## Matrix

In [ ]:
x = T.vector('x')
A = T.matrix('A')
z = A.dot(x)
normAx = theano.function([x, A], z.dot(z))

#### Compute this expression by yourself:

$A = [[1, 1], [1, 1]]$

$x = [0, 2]^T$

What is the answer?

Now compare it with theano result.

In [ ]:
normAx([0, 2], [[1, 1], [1, 1]])

In [ ]:
norm_grad = theano.function([x, A], T.grad(z.dot(z), x))

#### Compute gradient by yourself!!! Write its expression.

$(w, x) | grad(x) = w$

$(Ax, Ax) | grad(x) = 2 (A^T A) x$

#### Compute gradient for 
$A = [[0, 1], [1, 1]]$

**in point**:
$x = [1, 1]^T$

What is the answer?

Now compute the same with theano


In [ ]:
norm_grad([0, 1], [[1, 1], [1, 1]])

## Logistic regression 

Write logistic regression algorithm using theano! Now this is really very simple!

In [ ]:
# import function to create toy dataset for classification
from sklearn.datasets import make_blobs, make_moons
from sklearn.cross_validation import train_test_split 
from sklearn.metrics import roc_auc_score
from scipy.special import expit

In [ ]:
n_features = 10 # number of features
centers = 2 # number of classes
X, y = make_blobs(n_samples=10000, centers=centers, n_features=n_features)
X, X_test, y, y_test = train_test_split(X, y, train_size=0.5)

In [ ]:
# create theano objects for data(they should be shared because we know X, y)
X_ = theano.shared(X, name='X')
y_ = theano.shared(y, name='y')

In [ ]:
# define vector of weights, we don't know it, that is why:
w = T.vector(name='w')

#### Logistic regression description:
$p_i = sigmoid(\sum_j X_{ij}w_j)$

$loss=\sum y_i \log{p} + (1-y_i)\log{(1 - p)}$

$-loss \to min$

In [ ]:
# write expression for probabilities

In [ ]:
# write expression for loss

In [ ]:
# compile loss expression, compile gradient expression for loss
loss_function = ...
loss_grad = ...

Now we have expessions for loss and it gradient and we need some method of optimization

In [ ]:
from scipy.optimize import minimize

In [ ]:
# minimize loss function using its gradient
result = minimize(fun=loss_function, x0=numpy.zeros(n_features), jac=loss_grad)

In [ ]:
result

In [ ]:
w_optimal = result['x']

Now predict output of logistic regression for the test sample and compute AUC

In [ ]:
pred = X_test.dot(w_optimal)
roc_auc_score(y_test, pred)

####Is it simple with theano?

### Another implementation of logistic regression (add shift and l2 regularization)

In [ ]:
training_steps = 1000
X_ = T.matrix("x")
y_ = T.vector("y")
w = theano.shared(numpy.random.random(X.shape[1]), name="w")
# Add shift to the model
b = theano.shared(0., name="b")

In [ ]:
p_sig = T.nnet.sigmoid(T.dot(X_, w) - b) 
llh_ = y_ * T.log(p_sig) + (1 - y_) * T.log(1 - p_sig)
minus_llh_ = - llh_
cost = minus_llh_.mean() + 0.01*(w**2).sum() # add l2 regularization with parameter 0.01 / The cost to optimize
grad_w, grad_b = T.grad(cost, [w, b])
# define updates after run the function
train = theano.function(inputs=[X_, y_], outputs=[p_sig, minus_llh_], 
                        updates=[[w, w - 0.001*grad_w], [b, b - 0.001*grad_b]], name = "train")
predict = theano.function(inputs=[X_], outputs=p_sig, name = "predict")

for i in range(training_steps):
    probs, err = train(X, y)
pred = predict(X_test)
roc_auc_score(y_test, pred)

# Neural Network

## 1-hidden layer
This is a simple NN description with one hidden layer:

Params: 

* W,

* v

Calculations:

* h = sigmoid(X_.dot(W))
* output = v.dot(h)
* p_sig = sigmoid(output)
* p_bck = 1 - p_sig


Code it using theano

In [ ]:
...

## 2-hidden layers

### Using 1-hidden layer NN write 2-layers.

In [ ]:
# generate another dataset
X, y = make_moons(n_samples=20000)
X, X_test, y, y_test = train_test_split(X, y, train_size=0.5)

In [ ]:
scatter(X[:, 0], X[:, 1], c=y, alpha=0.01)

In [ ]:
...

### Define function fo NN to modify it by a simple way

In [ ]:
def fit_predict_NN(X, y, X_test, activate_functions, output_function, hidden_layers):
    X_ = theano.shared(X, name='X')
    y_ = theano.shared(y, name='y')
    param = T.vector()
    dim = X.shape[1]
    
    def activation(data_, parameter):
        n_previous = 0
        dim_previous = dim
        h = data_
        for n_hidden, func in zip(hidden_layers, activate_functions):
            N = dim_previous * n_hidden
            W_ = parameter[n_previous:n_previous + N].reshape((dim_previous, n_hidden))
            h = func(h.dot(W_))
            dim_previous = n_hidden
            n_previous += N

        # output     
        v_ = parameter[n_previous:]
        output = h.dot(v_)
        n_previous = n_previous + dim_previous
        
        return output_function(output), n_previous

    p_sig = activation(X_, param)[0]
    p_bck = 1 - p_sig
    llh_ = y_.dot(T.log(p_sig)) + (1 - y_).dot(T.log(p_bck))
    minus_llh_ = -llh_
    
    # optimize
    loss_function = theano.function([param], minus_llh_)
    loss_grad = theano.function([param], theano.grad(minus_llh_, param))
    result = minimize(loss_function, numpy.random.normal(size=activation(X_, param)[1]), jac=loss_grad)
    optimal_params = result['x']
    
    # predict data
    data = T.matrix()
    compiled_activation = theano.function([data, param], activation(data, param)[0])
    
    return compiled_activation(X_test, optimal_params)

In [ ]:
# generate another dataset
X, y = make_moons(n_samples=1000)
# add noise to data
X += numpy.random.random(size=X.shape[0] * X.shape[1]).reshape(X.shape)
X, X_test, y, y_test = train_test_split(X, y, train_size=0.5)

pred = fit_predict_NN(X, y, X_test, [T.nnet.sigmoid] * 3, T.nnet.sigmoid, [20, 10, 5])
roc_auc_score(y_test, pred)

## activation intermediate functions:
* sigmoid (which we used)
* relu 
* softplus

### ReLU - rectifier linear unit.

In the context of artificial neural networks, the rectifier is an activation function defined as

$f(x) = \max(0, x)$

where x is the input to a neuron. This activation function has been argued to be more biologically plausible than the widely used logistic sigmoid (which is inspired by probability theory; see logistic regression) and its more practical counterpart, the hyperbolic tangent. The rectifier is the most popular activation function for deep neural networks.

A unit employing the rectifier is also called a rectified linear unit (ReLU).

###Softplus
A smooth approximation to the rectifier is the analytic function

$f(x) = \ln(1 + e^x)$

which is called the softplus function.

In [ ]:
def generate_relu(alpha):
    def relu(x):
        return T.switch(x > 0, x, alpha * x)
    return relu

In [ ]:
# generate another dataset
X, y = make_moons(n_samples=1000)
X += numpy.random.random(size=X.shape[0] * X.shape[1]).reshape(X.shape)
X, X_test, y, y_test = train_test_split(X, y, train_size=0.5)

In [ ]:
pred = fit_predict_NN(X, y, X_test, [T.nnet.softplus] * 3, T.nnet.sigmoid, [10, 10, 5])
roc_auc_score(y_test, pred)

In [ ]:
pred = fit_predict_NN(X, y, X_test, [generate_relu(0.5)] * 2, T.nnet.sigmoid, [10, 10])
roc_auc_score(y_test, pred)

## Multiclassification  Problem

In this case we need use `softmax` activate fucntion for the last layer

**TODO**:

* Write multiclassification NN
* add shift to the NN formula ( h = (x, w) + b )

In [ ]:
def fit_predict_mult_NN(X, y, X_test, activate_functions, hidden_layers):
    X_ = theano.shared(X, name='X')
    y_ = theano.shared(y, name='y')
    param = T.vector()
    n_class = len(numpy.unique(y))
    dim = X.shape[1]
    
    def activation(data_, parameter):
        n_previous = 0
        dim_previous = dim
        h = data_
        for n_hidden, func in zip(hidden_layers, activate_functions):
            N = dim_previous * n_hidden
            W_ = parameter[n_previous:n_previous + N].reshape((dim_previous, n_hidden))
            n_previous += N
            b = parameter[n_previous: n_previous + n_hidden]
            n_previous += n_hidden
            h = func(h.dot(W_) + 0. * b[numpy.newaxis, :]) # here is brodcasing for b => it will be copied for each row
            dim_previous = n_hidden

        # output     
        v_ = parameter[n_previous:].reshape((dim_previous, n_class))
        output = h.dot(v_)
        n_previous = n_previous + dim_previous * n_class
        
        return T.nnet.softmax(output), n_previous
    
    p_ = activation(X_, param)[0]
    llh_ = T.log(p_[T.arange(len(X)), y_]).sum()
    minus_llh_ = -llh_
    
    # optimize
    loss_function = theano.function([param], minus_llh_)
    loss_grad = theano.function([param], theano.grad(minus_llh_, param))
    result = minimize(loss_function, numpy.random.normal(size=activation(X_, param)[1]), 
                      jac=loss_grad)
    optimal_params = result['x']
    
    data = T.matrix()
    compiled_activation = theano.function([data, param], activation(data, param)[0])

    return compiled_activation(X_test, optimal_params)   

In [ ]:
n_features = 20
n_class = 5
X, y = make_blobs(n_samples=2000, centers=n_class, n_features=n_features)
X += (numpy.random.normal(size=X.shape[0]*X.shape[1]) * 10).reshape(X.shape)
X, X_test, y, y_test = train_test_split(X, y, train_size=0.5)

In [ ]:
from sklearn.metrics import confusion_matrix
for func in [T.nnet.sigmoid, T.nnet.softmax, generate_relu(0.5)]:
    pred = fit_predict_mult_NN(X, y, X_test, [], [])
    imshow(confusion_matrix(y_test, numpy.argmax(pred, axis=1)), interpolation='nearest')
    plt.xticks(range(n_class), range(n_class))
    plt.yticks(range(n_class), range(n_class))
    show()

## Neural Net interafce in `hep_ml`: just write activation function using `theano`

In [ ]:
import hep_ml
from hep_ml import nnet

In [ ]:
from hep_ml.nnet import AbstractNeuralNetworkClassifier

class SimpleNeuralNetwork(AbstractNeuralNetworkClassifier):
    def prepare(self):
        # getting number of layers in input, hidden, output layers
        # note that we support only one hidden layer here
        n1, n2, n3 = self.layers_
        
        # creating parameters of neural network
        W1 = self._create_matrix_parameter('W1', n1, n2)
        W2 = self._create_matrix_parameter('W2', n2, n3)
        
        # defining activation function
        def activation(input):
            first = T.nnet.sigmoid(T.dot(input, W1))
            return T.dot(first, W2)

        return activation

In [ ]:
X, y = make_moons(n_samples=2000)
X += numpy.random.random(size=X.shape[0] * X.shape[1]).reshape(X.shape)
X, X_test, y, y_test = train_test_split(X, y, train_size=0.5)

In [ ]:
nnet_simple = SimpleNeuralNetwork()
nnet_simple.fit(X, y)
pred = nnet_simple.predict_proba(X_test)[:, 1]
print roc_auc_score(y_test, pred)

### Rewrite above binary classification NN into this interace 

Here is you daon't need to add `b` parameter, this interface does it and includes additional column in `X`

In [ ]:
class MyNeuralNetwork(AbstractNeuralNetworkClassifier):
    def prepare(self):
        # getting number of layers in input, hidden, output layers
        ...
        
        # defining activation function
        def activation(input):
            ...

        return activation

In [ ]:
nnet_my = MyNeuralNetwork(layers=[10, 5, 5])
nnet_my.fit(X, y)
pred = nnet_my.predict_proba(X_test)[:, 1]
print roc_auc_score(y_test, pred)

## Own rule

In [ ]:
# your ideas!

## NN in `hep_ml`

In [ ]:
from hep_ml.nnet import PairwiseNeuralNetwork, RBFNeuralNetwork, MLPClassifier

In [ ]:
for models in [PairwiseNeuralNetwork(), RBFNeuralNetwork(), MLPClassifier()]:
    models.fit(X, y)
    pred = models.predict_proba(X_test)[:, 1]
    print roc_auc_score(y_test, pred)

# Bagging over NN - meta algorithms

In [ ]:
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier

In [ ]:
base = RBFNeuralNetwork()
meta_ada = AdaBoostClassifier(base_estimator=base, n_estimators=10, learning_rate=0.05)
meta_ada.fit(X, y)
pred = meta_ada.predict_proba(X_test)[:, 1]
print roc_auc_score(y_test, pred)

In [ ]:
base = RBFNeuralNetwork()
meta_bagging = BaggingClassifier(base_estimator=base, n_estimators=50, max_samples=0.7)
meta_bagging.fit(X, y)
pred = meta_bagging.predict_proba(X_test)[:, 1]
print roc_auc_score(y_test, pred)

**Oy! This can really work for simple data! Try apply this meta algorithms possibilities to your analysis and improve your models!**